---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 2 - Introduction to NLTK

In part 1 of this assignment you will use nltk to explore the Herman Melville novel Moby Dick. Then in part 2 you will create a spelling recommender function that uses nltk to find words similar to the misspelling. 

## Part 1 - Analyzing Moby Dick

In [2]:
# Imports and downloads needed
# ---- Imports ---- #
import nltk
import pandas as pd
import numpy as np

# ---- Downloads ---- #
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lujan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lujan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lujan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
# If you would like to work with the raw text you can use 'moby_raw'
with open('moby.txt', 'r') as f:
    moby_raw = f.read()
    
# If you would like to work with the novel in nltk.Text format you can use 'text1'
moby_tokens = nltk.word_tokenize(moby_raw)
text1 = nltk.Text(moby_tokens)

### Example 1

How many tokens (words and punctuation symbols) are in text1?

*This function should return an integer.*

In [4]:
def example_one():
    
    return len(nltk.word_tokenize(moby_raw)) # or alternatively len(text1)

example_one()

255028

### Example 2

How many unique tokens (unique words and punctuation) does text1 have?

*This function should return an integer.*

In [5]:
def example_two():
    
    return len(set(nltk.word_tokenize(moby_raw))) # or alternatively len(set(text1))

example_two()

20742

### Example 3

After lemmatizing the verbs, how many unique tokens does text1 have?

*This function should return an integer.*

In [6]:
from nltk.stem import WordNetLemmatizer

def example_three():

    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(w,'v') for w in text1]

    return len(set(lemmatized))

example_three()

16887

### Question 1

What is the lexical diversity of the given text input? (i.e. ratio of unique tokens to the total number of tokens)

*This function should return a float.*

In [7]:
def answer_one():

    # Lenght of Tokens & Unique tokens
    lenUnique = len(set(text1))
    lenTokens = len(text1)
    
    # Define the ratio
    ratio = lenUnique / lenTokens

    return ratio

answer_one()

0.08133224587104161

### Question 2

What percentage of tokens is 'whale'or 'Whale'?

*This function should return a float.*

In [8]:
def answer_two():

    # Lenght of Tokens
    lenTokens = len(text1)
    
    # Filter the raw data just to have the tokens wanted
    whales = list(filter(lambda Token : (Token == 'Whale') or (Token == 'whale'), text1))
    
    # Lenght of Whales
    lenWhales = len(whales)

    # Define the ratio
    ratio = lenWhales / lenTokens

    return ratio

answer_two()

0.004125037250811676

### Question 3

What are the 20 most frequently occurring (unique) tokens in the text? What is their frequency?

*This function should return a list of 20 tuples where each tuple is of the form `(token, frequency)`. The list should be sorted in descending order of frequency.*

In [9]:
from nltk.probability import FreqDist

def answer_three():
    
    # Check the frequency for each unique Token
    freqToken = FreqDist(text1)
    freqs = freqToken.values()
    tokens = freqToken.keys()

    # Separate the first 20 tokens
    sortedTokens = [(token, freq) for freq, token in 
                    sorted(zip(freqs, tokens), reverse=True)]
                    
    firstTokens = sortedTokens[:20]

    return firstTokens

answer_three()

[(',', 19204),
 ('the', 13715),
 ('.', 7306),
 ('of', 6513),
 ('and', 6010),
 ('a', 4545),
 ('to', 4515),
 (';', 4173),
 ('in', 3908),
 ('that', 2978),
 ('his', 2459),
 ('it', 2196),
 ('I', 2113),
 ('!', 1767),
 ('is', 1722),
 ('--', 1713),
 ('with', 1659),
 ('he', 1658),
 ('was', 1639),
 ('as', 1620)]

### Question 4

What tokens have a length of greater than 5 and frequency of more than 150?

*This function should return an alphabetically sorted list of the tokens that match the above constraints. To sort your list, use `sorted()`*

In [10]:
def answer_four():
    
    # Tokenization of text and filtering
    freqToken = FreqDist(text1)
    tokens = freqToken.keys()
    freqs = freqToken.values()
    tokens = [(token, freq) for token, freq in zip(tokens, freqs)]

    # Filtering by conditions
    filtered = []

    for item in tokens:
        token, freq = item[0], item[-1]
        if len(token) > 5:
            if freq > 150:
                filtered += [token]

    # Sort of filtered
    sortedFiltered = sorted(filtered)
    return sortedFiltered

answer_four()

['Captain',
 'Pequod',
 'Queequeg',
 'Starbuck',
 'almost',
 'before',
 'himself',
 'little',
 'seemed',
 'should',
 'though',
 'through',
 'whales',
 'without']

### Question 5

Find the longest word in text1 and that word's length.

*This function should return a tuple `(longest_word, length)`.*

In [11]:
def answer_five():
    
    # Create a list of (word, lenght)
    wordLens = [len(word) for word in set(text1)]
    wordLen = [(word, lenght) for lenght, word 
                in sorted(zip(wordLens, set(text1)))]

    return wordLen[-1]

answer_five()

("twelve-o'clock-at-night", 23)

### Question 6

What unique words have a frequency of more than 2000? What is their frequency?

"Hint:  you may want to use `isalpha()` to check if the token is a word and not punctuation."

*This function should return a list of tuples of the form `(frequency, word)` sorted in descending order of frequency.*

In [17]:
def answer_six():
    
    # Define the frequency of the tokens
    freqToken = FreqDist(text1)
    freqs = freqToken.keys()
    tokens = freqToken.values()

    # Filter and reorganizing tokens
    tokenFreq = []

    for token, freq in zip(freqs, tokens):
        
        if token.isalpha():
            if freq > 2000:
                tokenFreq += [(freq, token)]

    return sorted(tokenFreq, reverse= True)

answer_six()

[(13715, 'the'),
 (6513, 'of'),
 (6010, 'and'),
 (4545, 'a'),
 (4515, 'to'),
 (3908, 'in'),
 (2978, 'that'),
 (2459, 'his'),
 (2196, 'it'),
 (2113, 'I')]

### Question 7

What is the average number of tokens per sentence?

*This function should return a float.*

In [19]:
from nltk.tokenize import sent_tokenize
def answer_seven():
    
    
    return sent_tokenize(moby_raw)

answer_seven()

['[Moby Dick by Herman Melville 1851]\n\n\nETYMOLOGY.',
 '(Supplied by a Late Consumptive Usher to a Grammar School)\n\nThe pale Usher--threadbare in coat, heart, body, and brain; I see him\nnow.',
 'He was ever dusting his old lexicons and grammars, with a queer\nhandkerchief, mockingly embellished with all the gay flags of all the\nknown nations of the world.',
 'He loved to dust his old grammars; it\nsomehow mildly reminded him of his mortality.',
 '"While you take in hand to school others, and to teach them by what\nname a whale-fish is to be called in our tongue leaving out, through\nignorance, the letter H, which almost alone maketh the signification\nof the word, you deliver that which is not true."',
 '--HACKLUYT\n\n"WHALE.',
 '... Sw. and Dan.',
 'HVAL.',
 'This animal is named from roundness\nor rolling; for in Dan.',
 'HVALT is arched or vaulted."',
 '--WEBSTER\'S\nDICTIONARY\n\n"WHALE.',
 '...',
 'It is more immediately from the Dut.',
 'and Ger.',
 'WALLEN;\nA.S. WALW-IAN,

### Question 8

What are the 5 most frequent parts of speech in this text? What is their frequency?

*This function should return a list of tuples of the form `(part_of_speech, frequency)` sorted in descending order of frequency.*

In [ ]:
def answer_eight():
    
    
    return # Your answer here

answer_eight()

## Part 2 - Spelling Recommender

For this part of the assignment you will create three different spelling recommenders, that each take a list of misspelled words and recommends a correctly spelled word for every word in the list.

For every misspelled word, the recommender should find find the word in `correct_spellings` that has the shortest distance*, and starts with the same letter as the misspelled word, and return that word as a recommendation.

*Each of the three different recommenders will use a different distance measure (outlined below).

Each of the recommenders should provide recommendations for the three default words provided: `['cormulent', 'incendenece', 'validrate']`.

In [21]:
from nltk.corpus import words

nltk.download('words')

correct_spellings = words.words()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\lujan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


### Question 9

For this recommender, your function should provide recommendations for the three default words provided above using the following distance metric:

**[Jaccard distance](https://en.wikipedia.org/wiki/Jaccard_index) on the trigrams of the two words.**

*This function should return a list of length three:
`['cormulent_reccomendation', 'incendenece_reccomendation', 'validrate_reccomendation']`.*

In [22]:
def answer_nine(entries=['cormulent', 'incendenece', 'validrate']):
    
    
    return # Your answer here
    
answer_nine()

### Question 10

For this recommender, your function should provide recommendations for the three default words provided above using the following distance metric:

**[Jaccard distance](https://en.wikipedia.org/wiki/Jaccard_index) on the 4-grams of the two words.**

*This function should return a list of length three:
`['cormulent_reccomendation', 'incendenece_reccomendation', 'validrate_reccomendation']`.*

In [ ]:
def answer_ten(entries=['cormulent', 'incendenece', 'validrate']):
    
    
    return # Your answer here
    
answer_ten()

### Question 11

For this recommender, your function should provide recommendations for the three default words provided above using the following distance metric:

**[Edit distance on the two words with transpositions.](https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance)**

*This function should return a list of length three:
`['cormulent_reccomendation', 'incendenece_reccomendation', 'validrate_reccomendation']`.*

In [ ]:
def answer_eleven(entries=['cormulent', 'incendenece', 'validrate']):
    
    
    return # Your answer here 
    
answer_eleven()